Tokenization of text to calculate OpenAI usage costs

Source: [How_to_count_tokens_with_tiktoken - OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

## Install titktoken

In [ ]:
import tiktoken

## Selection and loading of encoding

`cl100k_base` for models `gpt-4, gpt-3.5-turbo, text-embedding-ada-002`.

Refer to the [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb) for other types of models or changes in this encoding.


In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

## Tokenization of string

In [ ]:
encoding.encode("The puppy eats alone.")

In [ ]:
len(encoding.encode("The puppy eats alone."))

## Tokenize a series of messages

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens  

In [ ]:
example_messages = [
  {
    "role": "system",
    "content": "You are a customer and student support assistant for the online education platform in technology, English, and leadership called Platzi"
  },
  {
    "role": "user",
    "content": "How can I improve my content creation skills with artificial intelligence?"
  },
  {
    "role": "assistant",
    "content": "To improve your content creation skills with artificial intelligence, I suggest you take the Content Creation with Artificial Intelligence Workshop at https://platzi.com/cursos/contenido-ia/. In this course, you will learn how to use tools to generate content in an automated and efficient way. Don't miss it!"
  }
]

In [ ]:
num_tokens_from_messages(example_messages, model="gpt-3.5-turbo-0613")